In [1]:
#dependencias de intel sdk
import os
import shutil
import intelqsdk.cbindings
from openqasm_bridge.v2 import translate
from pennylane import numpy as np
import gc

#Pruebas en FULLSTATE simulator 

In [2]:
#rutas dentro del contenedor 
SDK_BASE = "/opt/intel/quantum-sdk/docker-intel_quantum_sdk_1.1.1.2024-11-15T22_03_32+00_00"  #ruta a los archivos bases del contenedor 
RUN_DIR = os.path.join(SDK_BASE, "quantum_runs")                                              #ruta a carpeta auxiliar 
WORKSPACE_DIR = "/workspace/test_simulador"                                                   #ruta al folio de proyecto
compiler_path = os.path.join(SDK_BASE, "intel-quantum-compiler")                              #ruta al compilador 
os.environ["LD_LIBRARY_PATH"] = f"{SDK_BASE}/lib:{SDK_BASE}/virtualenv/lib:{os.environ.get('LD_LIBRARY_PATH','')}"
os.chdir(RUN_DIR)
os.makedirs(RUN_DIR, exist_ok=True)                                                           #comprobacion de la existencia de la carpeta        

In [3]:

#circuito de ejemplo
qasm_example = """
OPENQASM 2.0;
qreg qubit_register[5];
h qubit_register[0];
cx qubit_register[0],qubit_register[1];
cx qubit_register[1],qubit_register[2];
cx qubit_register[2],qubit_register[3];
cx qubit_register[3],qubit_register[4];
"""

# Traductor  a C++ usando OpenQASM bridge
translated = translate(qasm_example, kernel_name='my_kernel')

cpp_path = os.path.join(RUN_DIR, "ghz.cpp")
with open(cpp_path, "w", encoding="utf8") as f:
    for line in translated:
        f.write(line + "\n")


In [ ]:
#ejecucion

sdk_name = "ghz"
intelqsdk.cbindings.compileProgram(compiler_path, "ghz.cpp", "-s", sdk_name)

iqs_config = intelqsdk.cbindings.IqsConfig()
iqs_config.num_qubits = 5
iqs_config.simulation_type = "noiseless"
iqs_device = intelqsdk.cbindings.FullStateSimulator(iqs_config)
iqs_device.ready()

intelqsdk.cbindings.callCppFunction("my_kernel", sdk_name)


# resultados
qbit_ref = intelqsdk.cbindings.RefVec()
for i in range(5):
    qbit_ref.append(intelqsdk.cbindings.QbitRef("qubit_register", i, sdk_name).get_ref())

probabilities = iqs_device.getProbabilities(qbit_ref)
intelqsdk.cbindings.FullStateSimulator.displayProbabilities(probabilities, qbit_ref)

zero_index = intelqsdk.cbindings.QssIndex("00000")
one_index = intelqsdk.cbindings.QssIndex("11111")
index_vec = intelqsdk.cbindings.QssIndexVec()
index_vec.append(zero_index)
index_vec.append(one_index)
probabilities_2 = iqs_device.getProbabilities(qbit_ref, index_vec, 0.1)

intelqsdk.cbindings.FullStateSimulator.displayProbabilities(probabilities_2)

for filename in os.listdir(RUN_DIR):
    try:
        os.remove(os.path.join(RUN_DIR, filename))
    except Exception:
        pass


del iqs_config,iqs_device 
gc.collect()


                       Intel(R) Quantum SDK (V1.1)
Processing source file:
/opt/intel/quantum-sdk/docker-intel_quantum_sdk_1.1.1.2024-11-15T22_03_32+00_00/quantum_runs/ghz.cpp 
-------------------------------------------------------------------------------
Intermediate representation (IR) output file: 
/opt/intel/quantum-sdk/docker-intel_quantum_sdk_1.1.1.2024-11-15T22_03_32+00_00/quantum_runs/ghz.ll
Generating quantum object file...
Compiling and linking to shared object...
Final shared object: 
/opt/intel/quantum-sdk/docker-intel_quantum_sdk_1.1.1.2024-11-15T22_03_32+00_00/quantum_runs/ghz.so
-------------------------------------------------------------------------------
Printing probability register of size 32
|00000⟩ : 0.5                           |10000⟩ : 2.298e-09                     
|01000⟩ : 1.986e-41                     |11000⟩ : 2.298e-09                     
|00100⟩ : 1.986e-41                     |10100⟩ : 7.561e-50                     
|01100⟩ : 1.986e-41               

In [15]:
#test_2 
np.random.seed(0)
params = np.random.uniform(0,np.pi,3)
parm1,parm2,parm3 = params 
print(f"parametros: rz({parm1:.2f}) , ry({parm2:.2f}) , rz({parm3:.2f})")

parametros: rz(1.72) , ry(2.25) , rz(1.89)


In [16]:
qasm_example_2 = f"""
OPENQASM 2.0;
include "qelib1.inc";
qreg qubit_register[1];

rz({parm1}) qubit_register[0];
ry({parm2}) qubit_register[0];
rz({parm3}) qubit_register[0];
"""
translated_2 = translate(qasm_example_2, kernel_name='rotaciones_kernel')

cpp_path_2 = os.path.join(RUN_DIR,"rotaciones.cpp")

with open(cpp_path_2, "w", encoding="utf8") as f:
    for line in translated_2:
        f.write(line + "\n")

In [ ]:
sdk_name_2 = "rotaciones"
intelqsdk.cbindings.compileProgram(compiler_path, "rotaciones.cpp", "-s", sdk_name_2)

iqs_config_2 = intelqsdk.cbindings.IqsConfig()
iqs_config_2.num_qubits = 1
iqs_config_2.simulation_type = "noiseless"
iqs_device_2 = intelqsdk.cbindings.FullStateSimulator(iqs_config_2)
iqs_device_2.ready()

# Llamar kernel generado
intelqsdk.cbindings.callCppFunction("rotaciones_kernel", sdk_name_2)

# resultados

qbit_ref_2 = intelqsdk.cbindings.RefVec()
qbit_ref_2.append(intelqsdk.cbindings.QbitRef("qubit_register", 0, sdk_name_2).get_ref())

probabilities_2 = iqs_device_2.getProbabilities(qbit_ref_2)
intelqsdk.cbindings.FullStateSimulator.displayProbabilities(probabilities_2, qbit_ref_2)

amps = iqs_device_2.getAmplitudes(qbit_ref_2)
intelqsdk.cbindings.FullStateSimulator.displayAmplitudes(amps, qbit_ref_2)


#limpieza

for filename in os.listdir(RUN_DIR):
    try:
        os.remove(os.path.join(RUN_DIR, filename))
    except Exception:
        pass


                       Intel(R) Quantum SDK (V1.1)
Processing source file:
/opt/intel/quantum-sdk/docker-intel_quantum_sdk_1.1.1.2024-11-15T22_03_32+00_00/quantum_runs/rotaciones.cpp 
-------------------------------------------------------------------------------
Intermediate representation (IR) output file: 
/opt/intel/quantum-sdk/docker-intel_quantum_sdk_1.1.1.2024-11-15T22_03_32+00_00/quantum_runs/rotaciones.ll
Generating quantum object file...
Compiling and linking to shared object...
Final shared object: 
/opt/intel/quantum-sdk/docker-intel_quantum_sdk_1.1.1.2024-11-15T22_03_32+00_00/quantum_runs/rotaciones.so
-------------------------------------------------------------------------------
Printing probability register of size 2
|0⟩   : 0.1872                        |1⟩   : 0.8128                        

Printing amplitude register of size 2
|0⟩   : (-0.102,-0.4204)              |1⟩   : (0.8983,0.07623)              

